In [1]:
from openai import OpenAI
import openai

from IPython.display import Markdown, display

import pyperclip

import os

import requests

import nbformat

In [22]:

def read_notebook(notebook_path):
    """
    Reads a Jupyter notebook file and returns its content.
    
    Args:
        notebook_path (str): Path to the .ipynb file
        
    Returns:
        nbformat.NotebookNode: Parsed notebook object
    """
    with open(notebook_path, 'r', encoding='utf-8') as file:
        return nbformat.read(file, as_version=4)

def extract_content(notebook):
    """
    Extracts text from code and markdown cells in a notebook.
    
    Args:
        notebook (nbformat.NotebookNode): The notebook to process
        
    Returns:
        str: Combined content from all cells
    """
    content_parts = []
    
    for cell in notebook.cells:
        if cell.cell_type == 'code':
            # Include both the code and any text outputs
            content_parts.append(f"# Code cell:\n{cell.source}\n")
            if 'outputs' in cell and cell.outputs:
                for output in cell.outputs:
                    if output.output_type == 'stream' and 'text' in output:
                        content_parts.append(f"# Output:\n{output.text}\n")
                    elif output.output_type == 'execute_result' and 'data' in output:
                        if 'text/plain' in output.data:
                            content_parts.append(f"# Result:\n{output.data['text/plain']}\n")
        elif cell.cell_type == 'markdown':
            content_parts.append(f"# Markdown:\n{cell.source}\n")
    
    return "\n".join(content_parts)

def summarize_with_openai(content, api_key, model="openai/gpt-4o"):
    """
    Sends notebook content to OpenAI API for summarization.
    
    Args:
        content (str): The notebook content to summarize
        api_key (str): Your OpenAI API key
        model (str): Which OpenAI model to use
        
    Returns:
        str: The generated summary
    """
    # Initialize the OpenAI client
    client = OpenAI(
    base_url="https://openrouter.ai/api/v1",
    api_key=api_key,
    )
    
    # Create the prompt for summarization
    prompt = f"""
    Please analyze the following Jupyter notebook content and provide a comprehensive summary.
    Organise your response under the following headings:
    1. The main purpose and objectives of the notebook
    2. Key code logic and functions implemented
    3. Important findings or results shown in outputs
    4. Overall structure and flow
    5. Instructions for use
    6. Theoretical Description of Methods

    Consistantly format the heading like "### 1. The main purpose and objectives of the notebook\n\n" 

    Provide a bibliograpghy of web references if used.

    Write the following mathematical expression using proper LaTeX syntax. Format it as an inline equation using 
    single dollar signs ($) or as a display equation using double dollar signs ($$) depending on the context. Do not escape backslashes.
    
    Be careful to avoid ParseError: KaTeX parse error: Expected 'EOF' errors when rendering equations 

    Avoid the over use of dot point or numbered lists.

    At the end of the document, add an interesting and possiblt relevant fact and give a reference to the fact if one is available 
    
    Notebook content:
    {content}
       """
    pyperclip.copy(prompt)
    system_prompt = 'You are an expert at analyzing and summarizing Jupyter notebooks for data science and programming contexts'

    response = requests.post(
        'https://openrouter.ai/api/alpha/responses',
        headers={
            'Authorization': f"Bearer {api_key}",
            'Content-Type': 'application/json',
        },
        json={
            'model': model,
            'input': [
                {
                    'type': 'message',
                    'role': 'system',
                    'content': [
                        {
                            'type': 'input_text',
                            'text': system_prompt,
                        },
                    ],
                },
                {
                    'type': 'message',
                    'role': 'user',
                    'content': [
                        {
                            'type': 'input_text',
                            'text': prompt,
                        },
                    ],
                },
            ],
            'plugins': [{'id': 'web', 'max_results': 10}],
            'max_output_tokens': 9000,
        }
    )
    result = response.json()
    
    return result['output'][0]['content'][0]['text']



def build_prompt(content):
    print(content)
   
    # Create the prompt for summarization
    prompt = f"""
    Please analyze the following Jupyter notebook content and provide a comprehensive summary.
    Organise your response under the following headings:
    1. The main purpose and objectives of the notebook
    2. Key code logic and functions implemented
    3. Important findings or results shown in outputs
    4. Overall structure and flow
    5. Instructions for use
    6. Theoretical Description of Methods

    Consistantly format the heading like "### 1. The main purpose and objectives of the notebook\n\n" 

    Provide a bibliograpghy of web references if used.

    Write the following mathematical expression using proper LaTeX syntax. Format it as an inline equation using 
    single dollar signs ($) or as a display equation using double dollar signs ($$) depending on the context. Do not escape backslashes.
    
    Be careful to avoid ParseError: KaTeX parse error: Expected 'EOF' errors when rendering equations 

    Avoid the over use of dot point or numbered lists.

    At the end of the document, add an interesting and possiblt relevant fact and give a reference to the fact if one is available 
    
    Notebook content:
    {content}
       """
    
    pyperclip.copy(prompt)


In [ ]:

def summarise_this(NOTEBOOK_PATH, model="google/gemini-2.5-flash-lite"):
    # Configuration
    OPENAI_API_KEY = os.environ.get("OPENROUTER_API_KEY")  # Get API key from environment

    #model="x-ai/grok-4-fast"
    #model="deepseek/deepseek-v3.2-exp"
    #model="google/gemini-2.5-flash-lite"
    #model="google/gemma-3n-e4b-it"
    #model="openai/gpt-5-mini"
    #model="inclusionai/ling-1t"


    if not OPENAI_API_KEY:
        raise ValueError("Please set the OPENAI_API_KEY environment variable")

    if not os.path.exists(NOTEBOOK_PATH):
        raise FileNotFoundError(f"Notebook file not found: {NOTEBOOK_PATH}")

    # Read and process the notebook
    print("Reading notebook...")
    notebook = read_notebook(NOTEBOOK_PATH)

    print("Extracting content...")
    notebook_content = extract_content(notebook)

    summary = summarize_with_openai(notebook_content, OPENAI_API_KEY, model=model)

    return summary

In [13]:
#model="x-ai/grok-4-fast"
#model="deepseek/deepseek-v3.2-exp"
#model="google/gemini-2.5-flash-lite"
#model="google/gemma-3n-e4b-it"
#model="openai/gpt-5-mini"
model="inclusionai/ling-1t"  
# $0.00327

summary = summarise_this('ai_script_summary.ipynb', model=model)

Markdown(summary)

Reading notebook...
Extracting content...


KeyboardInterrupt: 

In [195]:
pyperclip.copy(summary)

In [18]:
with open("output.md", "w") as file:
    file.write(summary)

---

In [24]:
note_book_contents = read_notebook('ai_script_summary.ipynb')
extracted = extract_content(note_book_contents)
build_prompt(extracted)


OSError: [Errno 22] Invalid argument: 'ai_script_summary.ipynb'

In [ ]:

    Please analyze the following Jupyter notebook content and provide a comprehensive summary.
    Organise your response under the following headings:
    1. The main purpose and objectives of the notebook
    2. Key code logic and functions implemented
    3. Important findings or results shown in outputs
    4. Overall structure and flow
    5. Instructions for use
    6. Theoretical Description of Methods

    Consistantly format the heading like "### 1. The main purpose and objectives of the notebook

" 

    Provide a bibliograpghy of web references if used.

    Write the following mathematical expression using proper LaTeX syntax. Format it as an inline equation using 
    single dollar signs ($) or as a display equation using double dollar signs ($$) depending on the context. Do not escape backslashes.

    Be careful to avoid ParseError: KaTeX parse error: Expected 'EOF' errors when rendering equations 

    Avoid the over use of dot point or numbered lists.

    At the end of the document, add an interesting and possiblt relevant fact and give a reference to the fact if one is available 

    Notebook content:
    # Code cell:
from openai import OpenAI
import openai

from IPython.display import Markdown, display

import pyperclip

import os

import requests

import nbformat

# Code cell:

def read_notebook(notebook_path):
    """
    Reads a Jupyter notebook file and returns its content.
    
    Args:
        notebook_path (str): Path to the .ipynb file
        
    Returns:
        nbformat.NotebookNode: Parsed notebook object
    """
    with open(notebook_path, 'r', encoding='utf-8') as file:
        return nbformat.read(file, as_version=4)

def extract_content(notebook):
    """
    Extracts text from code and markdown cells in a notebook.
    
    Args:
        notebook (nbformat.NotebookNode): The notebook to process
        
    Returns:
        str: Combined content from all cells
    """
    content_parts = []
    
    for cell in notebook.cells:
        if cell.cell_type == 'code':
            # Include both the code and any text outputs
            content_parts.append(f"# Code cell:\n{cell.source}\n")
            if 'outputs' in cell and cell.outputs:
                for output in cell.outputs:
                    if output.output_type == 'stream' and 'text' in output:
                        content_parts.append(f"# Output:\n{output.text}\n")
                    elif output.output_type == 'execute_result' and 'data' in output:
                        if 'text/plain' in output.data:
                            content_parts.append(f"# Result:\n{output.data['text/plain']}\n")
        elif cell.cell_type == 'markdown':
            content_parts.append(f"# Markdown:\n{cell.source}\n")
    
    return "\n".join(content_parts)

def summarize_with_openai(content, api_key, model="openai/gpt-4o"):
    """
    Sends notebook content to OpenAI API for summarization.
    
    Args:
        content (str): The notebook content to summarize
        api_key (str): Your OpenAI API key
        model (str): Which OpenAI model to use
        
    Returns:
        str: The generated summary
    """
    # Initialize the OpenAI client
    client = OpenAI(
    base_url="https://openrouter.ai/api/v1",
    api_key=api_key,
    )
    
    # Create the prompt for summarization
    prompt = f"""
    Please analyze the following Jupyter notebook content and provide a comprehensive summary.
    Organise your response under the following headings:
    1. The main purpose and objectives of the notebook
    2. Key code logic and functions implemented
    3. Important findings or results shown in outputs
    4. Overall structure and flow
    5. Instructions for use
    6. Theoretical Description of Methods

    Consistantly format the heading like "### 1. The main purpose and objectives of the notebook\n\n" 

    Provide a bibliograpghy of web references if used.

    Write the following mathematical expression using proper LaTeX syntax. Format it as an inline equation using 
    single dollar signs ($) or as a display equation using double dollar signs ($$) depending on the context. Do not escape backslashes.
    
    Be careful to avoid ParseError: KaTeX parse error: Expected 'EOF' errors when rendering equations 

    Avoid the over use of dot point or numbered lists.

    At the end of the document, add an interesting and possiblt relevant fact and give a reference to the fact if one is available 
    
    Notebook content:
    {content}
       """
    pyperclip.copy(prompt)
    system_prompt = 'You are an expert at analyzing and summarizing Jupyter notebooks for data science and programming contexts'

    response = requests.post(
        'https://openrouter.ai/api/alpha/responses',
        headers={
            'Authorization': f"Bearer {api_key}",
            'Content-Type': 'application/json',
        },
        json={
            'model': model,
            'input': [
                {
                    'type': 'message',
                    'role': 'system',
                    'content': [
                        {
                            'type': 'input_text',
                            'text': system_prompt,
                        },
                    ],
                },
                {
                    'type': 'message',
                    'role': 'user',
                    'content': [
                        {
                            'type': 'input_text',
                            'text': prompt,
                        },
                    ],
                },
            ],
            'plugins': [{'id': 'web', 'max_results': 10}],
            'max_output_tokens': 9000,
        }
    )
    result = response.json()
    
    return result['output'][0]['content'][0]['text']



def build_prompt(content):
   
    # Create the prompt for summarization
    prompt = f"""
    Please analyze the following Jupyter notebook content and provide a comprehensive summary.
    Organise your response under the following headings:
    1. The main purpose and objectives of the notebook
    2. Key code logic and functions implemented
    3. Important findings or results shown in outputs
    4. Overall structure and flow
    5. Instructions for use
    6. Theoretical Description of Methods

    Consistantly format the heading like "### 1. The main purpose and objectives of the notebook\n\n" 

    Provide a bibliograpghy of web references if used.

    Write the following mathematical expression using proper LaTeX syntax. Format it as an inline equation using 
    single dollar signs ($) or as a display equation using double dollar signs ($$) depending on the context. Do not escape backslashes.
    
    Be careful to avoid ParseError: KaTeX parse error: Expected 'EOF' errors when rendering equations 

    Avoid the over use of dot point or numbered lists.

    At the end of the document, add an interesting and possiblt relevant fact and give a reference to the fact if one is available 
    
    Notebook content:
    {content}
       """
    
    pyperclip.copy(prompt)


# Code cell:

def summarise_this(NOTEBOOK_PATH, model="x-ai/grok-4-fast"):
    # Configuration
    OPENAI_API_KEY = os.environ.get("OPENROUTER_API_KEY")  # Get API key from environment

    #model="x-ai/grok-4-fast"
    #model="deepseek/deepseek-v3.2-exp"
    #model="google/gemini-2.5-flash-lite"
    #model="google/gemma-3n-e4b-it"
    #model="openai/gpt-5-mini"


    if not OPENAI_API_KEY:
        raise ValueError("Please set the OPENAI_API_KEY environment variable")

    if not os.path.exists(NOTEBOOK_PATH):
        raise FileNotFoundError(f"Notebook file not found: {NOTEBOOK_PATH}")

    # Read and process the notebook
    print("Reading notebook...")
    notebook = read_notebook(NOTEBOOK_PATH)

    print("Extracting content...")
    notebook_content = extract_content(notebook)

    summary = summarize_with_openai(notebook_content, OPENAI_API_KEY, model=model)

    return summary

# Code cell:
  
# $0.00327

summary = summarise_this('conformal_prediction.ipynb')

Markdown(summary)

# Output:
Reading notebook...
Extracting content...


# Result:
<IPython.core.display.Markdown object>

# Code cell:
summary = summarise_this('conformal_prediction.ipynb', model="qwen/qwen3-235b-a22b:free")

Markdown(summary)

# Output:
Reading notebook...
Extracting content...


# Result:
<IPython.core.display.Markdown object>

# Code cell:
pyperclip.copy(summary)

# Code cell:
with open("output.md", "w") as file:
    file.write(summary)

# Markdown:
---

# Code cell:
summary5 = summarise_this('TopologicalQueries.ipynb', model="alibaba/tongyi-deepresearch-30b-a3b:free")

Markdown(summary5)

# Output:
Reading notebook...
Extracting content...


# Code cell:
summary6 = summarise_this('TopologicalQueries.ipynb', model="qwen/qwen3-235b-a22b:free")

Markdown(summary6)

# Output:
Reading notebook...
Extracting content...


# Result:
<IPython.core.display.Markdown object>

# Code cell:
summary = build_prompt('conformal_prediction.ipynb')

# Code cell:
def get_prompt(NOTEBOOK_PATH):
    # Configuration


    # Read and process the notebook
    print("Reading notebook...")
    notebook = read_notebook(NOTEBOOK_PATH)

    print("Extracting content...")
    notebook_content = extract_content(notebook)

    build_prompt(notebook_content)


# Code cell:
get_prompt('conformal_prediction.ipynb')

# Output:
Reading notebook...
Extracting content...


# Code cell:

    Please analyze the following Jupyter notebook content and provide a comprehensive summary.
    Organise your response under the following headings:
    1. The main purpose and objectives of the notebook
    2. Key code logic and functions implemented
    3. Important findings or results shown in outputs
    4. Overall structure and flow
    5. Instructions for use
    6. Theoretical Description of Methods

    Consistantly format the heading like "### 1. The main purpose and objectives of the notebook

" 

    Provide a bibliograpghy of web references if used.

    Write the following mathematical expression using proper LaTeX syntax. Format it as an inline equation using 
    single dollar signs ($) or as a display equation using double dollar signs ($$) depending on the context. Do not escape backslashes.

    Be careful to avoid ParseError: KaTeX parse error: Expected 'EOF' errors when rendering equations 

    Avoid the over use of dot point or numbered lists.

    At the end of the document, add an interesting and possiblt relevant fact and give a reference to the fact if one is available 

    Notebook content:
    conformal_prediction.ipynb
       

       

'# Code cell:\nfrom openai import OpenAI\nimport openai\n\nfrom IPython.display import Markdown, display\n\nimport pyperclip\n\nimport os\n\nimport requests\n\nimport nbformat\n\n# Code cell:\n\ndef read_notebook(notebook_path):\n    """\n    Reads a Jupyter notebook file and returns its content.\n    \n    Args:\n        notebook_path (str): Path to the .ipynb file\n        \n    Returns:\n        nbformat.NotebookNode: Parsed notebook object\n    """\n    with open(notebook_path, \'r\', encoding=\'utf-8\') as file:\n        return nbformat.read(file, as_version=4)\n\ndef extract_content(notebook):\n    """\n    Extracts text from code and markdown cells in a notebook.\n    \n    Args:\n        notebook (nbformat.NotebookNode): The notebook to process\n        \n    Returns:\n        str: Combined content from all cells\n    """\n    content_parts = []\n    \n    for cell in notebook.cells:\n        if cell.cell_type == \'code\':\n            # Include both the code and any text outpu

> ## <strong style="color:#00b8d4; font-size:28px;">AI Script Summary</strong>
> <span style="color:#757575; font-size:18px; display:block; margin-top:1px;">Use LLMs to generate summaries of Jupyter notebook 
> scripts </span>
>
> <strong>Authors:</strong> F. R. Bennett &nbsp;&nbsp; <br/><br/>
> <strong>Date:</strong> 17/10/25  &nbsp;&nbsp; <br/><br/>
> <strong>Version:</strong> 1.0

